# Simple Rounding example

Simple Rounding is a heuristic, that rounds each integer variable to the closest integer of the relaxation LP solution.

## Installing the packages

In the following we use the `GurobiSolver` and thus need to install gurobi.

In [ ]:
%%capture
!pip install gurobipy

Afterwards, we can install hips.

In [ ]:
%%capture
!pip install https://github.com/cxlvinchau/hips/archive/master.zip

## Example

First, we need to import the dependencies.

In [ ]:
from hips import VarTypes, HIPSArray, ProblemSense
from hips.models import MIPModel
from hips.solver import GurobiSolver
from hips.heuristics import SimpleRounding

Next we initialize a helper function, which allows us to load a model. The model represents the following MIP:
$$
\begin{array}{lr@{}c@{}r@{}l}
    \text{maximize }   & x_1 + x_2  \\
    \text{subject to } & x_1 + \frac{2}{3} x_2 \leq 2 \\
                       & x_1 + \frac{3}{2} x_2 \leq 3 \\
                       & x_1, x_2 \geq 0 \\
                       & x_1, x_2 \in \mathbb{Z}
\end{array}
$$

In [ ]:
def build_model(mip_model):
    x = mip_model.add_variable("x", VarTypes.INTEGER, lb=0, ub=float("inf"), dim=2)
    constr1 = HIPSArray([1,2/3])*x <= 2
    constr2 = HIPSArray([1,1.5])*x <= 3
    mip_model.add_constraint(constr1)
    mip_model.add_constraint(constr2)
    obj_func = HIPSArray([1,1])*x
    mip_model.set_objective(obj_func)
    mip_model.lp_model.set_lp_sense(ProblemSense.MAX)

We can now build the model:

In [ ]:
mip_model = MIPModel(GurobiSolver())
build_model(mip_model)

Finally we setup the heuristic and start the computation:

In [ ]:
heur = SimpleRounding(mip_model)
heur.compute()
print("Status: {}".format(heur.get_status()))
print("Found solution: {}".format(heur.get_objective_value()))
print("With Variable values: {}".format({var: heur.variable_solution(var) for var in mip_model.get_variables()}))